<a href="https://colab.research.google.com/github/boyu571/kpc_lecture_notes/blob/master/ABC%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IRR NPV 사례

In [ ]:
!pip install numpy-financial

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy_financial as npf
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rcParams['font.family'] = 'NanumGothic'

# from google.colab import files
# myfile = files.upload()

Sudo가 이 컴퓨터에서 사용하지 않도록 설정되어 있습니다. 사용하도록 설정하려면 으로 이동하세요. ]8;;ms-settings:developers\Developer Settings page]8;;\ 설정 앱의
Sudo가 이 컴퓨터에서 사용하지 않도록 설정되어 있습니다. 사용하도록 설정하려면 으로 이동하세요. ]8;;ms-settings:developers\Developer Settings page]8;;\ 설정 앱의
'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
# 1. CSV 읽기
df1 = pd.read_csv('C4_data.csv', index_col=0)
df1

,‘21,‘22,‘23,‘24,‘25,‘26,‘27,‘28,‘29,‘30
항목,,,,,,,,,,
날짜,2021-01-01,2022-01-01,2023-01-01,2024-01-01,2025-01-01,2026-01-01,2027-01-01,2028-01-01,2029-01-01,2030-01-01
년차,1,2,3,4,5,6,7,8,9,10
매출액(A),8856,11513,14967,19457,25294,32882,42746,55570,72241,93913
비용(B),5303,6629,8286,10358,12948,16185,20231,25288,31610,39513


In [4]:
# 2. 전치 및 열 이름 정리
df1_t = df1.T
df1_t.columns = df1_t.columns.str.strip()
df1_t

항목,날짜,년차,매출액(A),비용(B)
‘21,2021-01-01,1,8856,5303
‘22,2022-01-01,2,11513,6629
‘23,2023-01-01,3,14967,8286
‘24,2024-01-01,4,19457,10358
‘25,2025-01-01,5,25294,12948
‘26,2026-01-01,6,32882,16185
‘27,2027-01-01,7,42746,20231
‘28,2028-01-01,8,55570,25288
‘29,2029-01-01,9,72241,31610
‘30,2030-01-01,10,93913,39513


In [5]:
# 3. 날짜 정보 추출 (행 또는 열에 따라 처리)
if '날짜' in df1_t.index:
    date_row = df1_t.loc['날짜']
    date_list = [datetime.strptime(str(d).strip(), '%Y-%m-%d') for d in date_row.tolist()]
    df1_t = df1_t.drop(index='날짜')  # 행 제거
elif '날짜' in df1_t.columns:
    date_col = df1_t['날짜']
    date_list = [datetime.strptime(str(d).strip(), '%Y-%m-%d') for d in date_col.tolist()]
    df1_t = df1_t.drop(columns='날짜')  # 열 제거
else:
    date_list = []

date_list

[datetime.datetime(2021, 1, 1, 0, 0),
 datetime.datetime(2022, 1, 1, 0, 0),
 datetime.datetime(2023, 1, 1, 0, 0),
 datetime.datetime(2024, 1, 1, 0, 0),
 datetime.datetime(2025, 1, 1, 0, 0),
 datetime.datetime(2026, 1, 1, 0, 0),
 datetime.datetime(2027, 1, 1, 0, 0),
 datetime.datetime(2028, 1, 1, 0, 0),
 datetime.datetime(2029, 1, 1, 0, 0),
 datetime.datetime(2030, 1, 1, 0, 0)]

In [6]:
# 4. 숫자형 변환 (콤마 제거 후 float 처리)
for col in df1_t.columns:
    df1_t[col] = df1_t[col].astype(str).str.replace(',', '').astype(float)

df1_t

항목,년차,매출액(A),비용(B)
‘21,1.0,8856.0,5303.0
‘22,2.0,11513.0,6629.0
‘23,3.0,14967.0,8286.0
‘24,4.0,19457.0,10358.0
‘25,5.0,25294.0,12948.0
‘26,6.0,32882.0,16185.0
‘27,7.0,42746.0,20231.0
‘28,8.0,55570.0,25288.0
‘29,9.0,72241.0,31610.0
‘30,10.0,93913.0,39513.0


In [7]:
# 5. 계산
df1_t['영업이익(C)'] = df1_t['매출액(A)'] - df1_t['비용(B)']
df1_t['법인세(D)'] = (df1_t['영업이익(C)'] * 0.25).round(0)
df1_t['세후 순 영업 이익(C-D)'] = df1_t['영업이익(C)'] - df1_t['법인세(D)'].round(0)
df1_t

항목,년차,매출액(A),비용(B),영업이익(C),법인세(D),세후 순 영업 이익(C-D)
‘21,1.0,8856.0,5303.0,3553.0,888.0,2665.0
‘22,2.0,11513.0,6629.0,4884.0,1221.0,3663.0
‘23,3.0,14967.0,8286.0,6681.0,1670.0,5011.0
‘24,4.0,19457.0,10358.0,9099.0,2275.0,6824.0
‘25,5.0,25294.0,12948.0,12346.0,3086.0,9260.0
‘26,6.0,32882.0,16185.0,16697.0,4174.0,12523.0
‘27,7.0,42746.0,20231.0,22515.0,5629.0,16886.0
‘28,8.0,55570.0,25288.0,30282.0,7570.0,22712.0
‘29,9.0,72241.0,31610.0,40631.0,10158.0,30473.0
‘30,10.0,93913.0,39513.0,54400.0,13600.0,40800.0


In [8]:
# 6. 초기 변수
discount_rate = 0.059
initial_investment = 40000

In [9]:
# 7. NPV
df1_t['NPV(i=5.9%)'] = [
    cf / ((1 + discount_rate) ** (i + 1))
    for i, cf in enumerate(df1_t['세후 순 영업 이익(C-D)'])
]
df1_t['누적 NPV(i=5.9%)'] = df1_t['NPV(i=5.9%)'].cumsum().round(0)
df1_t['투자회수'] = df1_t['누적 NPV(i=5.9%)'] - initial_investment
df1_t

항목,년차,매출액(A),비용(B),영업이익(C),법인세(D),세후 순 영업 이익(C-D),NPV(i=5.9%),누적 NPV(i=5.9%),투자회수
‘21,1.0,8856.0,5303.0,3553.0,888.0,2665.0,2516.525024,2517.0,-37483.0
‘22,2.0,11513.0,6629.0,4884.0,1221.0,3663.0,3266.216726,5783.0,-34217.0
‘23,3.0,14967.0,8286.0,6681.0,1670.0,5011.0,4219.262274,10002.0,-29998.0
‘24,4.0,19457.0,10358.0,9099.0,2275.0,6824.0,5425.692514,15428.0,-24572.0
‘25,5.0,25294.0,12948.0,12346.0,3086.0,9260.0,6952.342934,22380.0,-17620.0
‘26,6.0,32882.0,16185.0,16697.0,4174.0,12523.0,8878.357324,31258.0,-8742.0
‘27,7.0,42746.0,20231.0,22515.0,5629.0,16886.0,11304.596466,42563.0,2563.0
‘28,8.0,55570.0,25288.0,30282.0,7570.0,22712.0,14357.793362,56921.0,16921.0
‘29,9.0,72241.0,31610.0,40631.0,10158.0,30473.0,18190.790437,75112.0,35112.0
‘30,10.0,93913.0,39513.0,54400.0,13600.0,40800.0,22998.555629,98110.0,58110.0


In [10]:
# 8. 투자 회수 시점
payback_candidates = df1_t[df1_t['투자회수'] >= 0]
payback_label = payback_candidates.index[0] if not payback_candidates.empty else None
print("투자 회수 시점 :", payback_label)

투자 회수 시점 : ‘27


In [11]:
# 9. XNPV 함수
def xnpv(rate, values, dates):
    base = dates[0]
    return sum(val / (1 + rate) ** ((d - base).days / 365.0) for val, d in zip(values, dates))

In [12]:
# 10. XNPV 계산
xnpv_year = 6

if date_list:
    values = df1_t['세후 순 영업 이익(C-D)'].iloc[:xnpv_year].tolist()
    dates = date_list[:xnpv_year]
    xnpv_result = round(xnpv(discount_rate, values, dates), 2)
    print("XNPV :", xnpv_result)
else:
    print("날짜 정보 없음")

XNPV : 33100.01


In [13]:
# 11. IRR 계산용 cashflow 리스트 구성 (0년차 투자 포함)
irr_year = 7 
irr_values = [-initial_investment] + df1_t['세후 순 영업 이익(C-D)'].iloc[:irr_year].tolist()

# IRR 계산
irr_result = npf.irr(irr_values)

# 결과 출력
print(f"IRR : {irr_result * 100:.2f}%")

IRR : 7.24%
